In [6]:
# Imports from the other scripts
from riskInput import get_risk_profile
from quantify_risk import quantify_risk

In [7]:
import pandas as pd
import numpy as np
import os

In [41]:
def prepare_optimization_inputs(files):
    """
    Prepare the inputs for portfolio optimization: Expected Returns and Covariance Matrix.
    
    Parameters:
    - files (list): List of file paths to CSV files containing stock data with a 'Log Return' column.

    Returns:
    - expected_returns (pd.Series): Historical mean log returns for each asset.
    - covariance_matrix (pd.DataFrame): Covariance matrix of the log returns for the assets.
    - risk_targets (dict): Target metrics for volatility, drawdown, and Sharpe Ratio based on risk level.
    """
    # Get the user's risk profile and quantify it
    try:
        risk_profile = get_risk_profile()
        risk_targets = quantify_risk(risk_profile)
    except ValueError as e:
        print(e)
        return None, None, None

    print(f"\nPreparing optimization inputs for {risk_profile.capitalize()} Risk Profile.\n")
    
    # DataFrame to hold log returns of each asset
    log_returns_df = pd.DataFrame()

    # Calculate log returns for each asset
    for file_path in files:
        df = pd.read_csv(file_path)

        # Calculate the log returns if not already present
        if 'Log Return' not in df.columns:
            df['Log Return'] = np.log(df['Adj Close'] / df['Adj Close'].shift(1))
        
        # Extract stock name from file path and add its log returns to the DataFrame
        stock_name = file_path.split("/")[-1].replace('.csv', '')
        log_returns_df[stock_name] = df['Log Return']

    # Calculate expected returns (mean log returns) for each asset
    expected_returns = log_returns_df.mean()

    # Calculate the covariance matrix of the log returns
    covariance_matrix = log_returns_df.cov()

    # Debug statement to check the initial counts
    print("Initial Assets:", expected_returns.index.tolist())

    # Adjust based on risk profile if low risk
    if risk_profile == "low":
        # Calculate annualized volatilities
        volatilities = log_returns_df.std() * np.sqrt(252)  # 252 trading days in a year
        max_volatility = risk_targets['volatility_target']

        # Filter based on the max volatility allowed for low-risk profile
        filtered_assets = volatilities[volatilities <= max_volatility].index
        print("Filtered Assets (within volatility target):", filtered_assets.tolist())

        # Update expected returns and covariance matrix based on the filtered assets
        expected_returns = expected_returns[filtered_assets]
        covariance_matrix = covariance_matrix.loc[filtered_assets, filtered_assets]

    # Final debug statement to confirm results
    print("\nFinal Expected Returns:")
    print(expected_returns)
    print("\nFinal Covariance Matrix:")
    print(covariance_matrix)
    
    return expected_returns, covariance_matrix, risk_targets

In [11]:
# Example usage
files = ['../Fetch_data/data/Extra10/AMZN_2020_2024.csv', '../Fetch_data/data/Extra10/GOOGL_2020_2024.csv']  # Sample file paths
expected_returns, covariance_matrix, risk_targets = prepare_optimization_inputs(files)


Preparing optimization inputs for High Risk Profile.

Initial Assets: ['AMZN_2020_2024', 'GOOGL_2020_2024']

Final Expected Returns:
AMZN_2020_2024     0.000622
GOOGL_2020_2024    0.000634
dtype: float64

Final Covariance Matrix:
                 AMZN_2020_2024  GOOGL_2020_2024
AMZN_2020_2024         0.000496         0.000295
GOOGL_2020_2024        0.000295         0.000388


<h1> Passing all the files of 40 Stocks

In [8]:
def load_stock_data(folder_paths):
    """
    Load stock data from multiple folders and calculate log returns.

    Parameters:
    - folder_paths (list): List of paths to folders containing CSV files with stock data.

    Returns:
    - log_returns_df (pd.DataFrame): DataFrame containing log returns of all assets.
    """
    log_returns_df = pd.DataFrame()
    
    # Loop through each folder and load all CSV files
    for folder in folder_paths:
        for file_name in os.listdir(folder):
            if file_name.endswith('.csv'):
                file_path = os.path.join(folder, file_name)
                df = pd.read_csv(file_path)
                
                # Calculate log returns if not already present
                if 'Log Return' not in df.columns:
                    df['Log Return'] = np.log(df['Adj Close'] / df['Adj Close'].shift(1))
                
                # Extract stock name from file name and add its log returns to the DataFrame
                stock_name = file_name.replace('.csv', '')
                log_returns_df[stock_name] = df['Log Return']
    
    return log_returns_df

In [13]:
def prepare_optimization_inputs(folder_paths):
    """
    Prepare the inputs for portfolio optimization: Expected Returns and Covariance Matrix.
    
    Parameters:
    - folder_paths (list): List of paths to folders containing CSV files with stock data.

    Returns:
    - expected_returns (pd.Series): Historical mean log returns for each asset.
    - covariance_matrix (pd.DataFrame): Covariance matrix of the log returns for the assets.
    - risk_targets (dict): Target metrics for volatility, drawdown, and Sharpe Ratio based on risk level.
    """
    # Get the user's risk profile and quantify it
    try:
        risk_profile = get_risk_profile()
        risk_targets = quantify_risk(risk_profile)
    except ValueError as e:
        print(e)
        return None, None, None

    print(f"\nPreparing optimization inputs for {risk_profile.capitalize()} Risk Profile.\n")
    
    # Load stock data from the specified folders
    log_returns_df = load_stock_data(folder_paths)

    # Calculate expected returns (mean log returns) for each asset
    expected_returns = log_returns_df.mean()

    # Calculate the covariance matrix of the log returns
    covariance_matrix = log_returns_df.cov()

    # Calculate annualized volatilities for additional filtering
    volatilities = log_returns_df.std() * np.sqrt(252)  # Annualized volatility
    sharpe_ratios = expected_returns / volatilities

    # Print initial assets count
    print("Initial Assets:", expected_returns.index.tolist())
    print(f"Total Initial Assets: {len(expected_returns.index)}")

    # Apply less strict selection based on risk profile
    if risk_profile == "low":
        max_volatility = risk_targets['volatility_target'] * 1.5  # Increase threshold to be more lenient
        filtered_assets = volatilities[volatilities <= max_volatility].index

    # If still no assets are selected, include the lowest volatility assets as a baseline
        if len(filtered_assets) == 0:
            filtered_assets = volatilities.nsmallest(5).index  # Select top 5 by lowest volatility as a fallback

    elif risk_profile == "medium":
        max_volatility = risk_targets['volatility_target'] * 1.2  # Increase threshold moderately
        filtered_assets = volatilities[volatilities <= max_volatility].index

    # Updated filter: Select only top 80% by Sharpe ratio
        top_assets_by_sharpe = sharpe_ratios[filtered_assets].nlargest(len(filtered_assets) * 8 // 10).index
        filtered_assets = top_assets_by_sharpe

    # If no assets remain, include a baseline selection of lowest volatility assets
        if len(filtered_assets) == 0:
            filtered_assets = volatilities.nsmallest(10).index  # Select top 10 by lowest volatility as a fallback

    elif risk_profile == "high":
        max_volatility = risk_targets['volatility_target'] * 1.3  # Increase threshold further for high risk
        filtered_assets = volatilities[volatilities <= max_volatility].index

    # Updated filter: Select top 90% by expected return to capture more assets
        top_assets_by_return = expected_returns[filtered_assets].nlargest(len(filtered_assets) * 9 // 10).index
        filtered_assets = top_assets_by_return

    # Final fallback: If still too few assets, select a baseline of lowest volatility assets
        if len(filtered_assets) < 10:
            filtered_assets = volatilities.nsmallest(15).index  # Ensure at least 15 assets by lowest volatility
 

    # Updated filter: Select only top 50% by expected return
    top_assets_by_return = expected_returns[filtered_assets].nlargest(len(filtered_assets) * 5 // 10).index
    filtered_assets = top_assets_by_return

    # Update expected returns and covariance matrix based on the filtered assets
    expected_returns = expected_returns[filtered_assets]
    covariance_matrix = covariance_matrix.loc[filtered_assets, filtered_assets]

    # Final asset count after filtering
    print(f"\nTotal Selected Assets for {risk_profile.capitalize()} Risk Profile: {len(expected_returns.index)}")

    # Display final outputs for verification
    print("\nFinal Expected Returns:")
    print(expected_returns)
    print("\nFinal Covariance Matrix:")
    print(covariance_matrix)
    
    return expected_returns, covariance_matrix, risk_targets

# Example usage
folder_paths = ['../Fetch_data/data/US-30/', '../Fetch_data/data/Extra10/']  # Paths to the two folders
expected_returns, covariance_matrix, risk_targets = prepare_optimization_inputs(folder_paths)


Preparing optimization inputs for High Risk Profile.

Initial Assets: ["McDonald's_MCD_2020_2024", 'Cisco Systems_CSCO_2020_2024', 'Nike_NKE_2020_2024', 'American Express_AXP_2020_2024', 'Merck_MRK_2020_2024', 'Disney_DIS_2020_2024', 'Johnson & Johnson_JNJ_2020_2024', 'Coca-Cola_KO_2020_2024', 'Verizon_VZ_2020_2024', 'Salesforce_CRM_2020_2024', 'Walmart_WMT_2020_2024', 'Microsoft_MSFT_2020_2024', 'UnitedHealth Group_UNH_2020_2024', 'The Travelers Companies_TRV_2020_2024', 'Honeywell_HON_2020_2024', 'Walgreens Boots Alliance_WBA_2020_2024', 'Amgen_AMGN_2020_2024', 'Goldman Sachs_GS_2020_2024', 'Dow Inc._DOW_2020_2024', 'Caterpillar_CAT_2020_2024', 'Visa_V_2020_2024', 'Procter & Gamble_PG_2020_2024', 'JPMorgan Chase_JPM_2020_2024', '3M_MMM_2020_2024', 'Home Depot_HD_2020_2024', 'Chevron_CVX_2020_2024', 'Apple_AAPL_2020_2024', 'IBM_IBM_2020_2024', 'Intel_INTC_2020_2024', 'Boeing_BA_2020_2024', 'V_2020_2024', 'TSLA_2020_2024', 'PFE_2020_2024', 'AMZN_2020_2024', 'NFLX_2020_2024', 'BRK-B_20